<a href="https://colab.research.google.com/github/pangxiangyang/StockPricePrediction/blob/master/Data_Processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
for dir_name in dirlist:
  
  os.chdir(dir_name) # change directory from working dir to dir with files
  unzipped_dir = dir_name + "/Unzipped/" 
  print( unzipped_dir)
  
  if( not os.path.isdir( unzipped_dir ) ):
    os.makedirs( unzipped_dir)

  for item in os.listdir(dir_name): # loop through items in dir
      if item.endswith(extension): # check for ".zip" extension
        file_name = os.path.abspath(item) # get full path of files
        zip_ref = zipfile.ZipFile(file_name) # create zipfile object
        zip_ref.extractall(unzipped_dir) # extract file to dir
        zip_ref.close() # close file
        #os.remove(file_name) # delete zipped file

In [0]:
import pandas as pd


unzipped_dir = '/content/drive/My Drive/NewData/8877.MY/Unzipped/'  #ekovest
#unzipped_dir = '/content/drive/My Drive/NewData/5222.MY/Unzipped/'  #fgv
#unzipped_dir = '/content/drive/My Drive/NewData/4715.MY/Unzipped/'  #genm
#unzipped_dir = '/content/drive/My Drive/NewData/4723.MY/Unzipped/'  #jaks
#unzipped_dir = '/content/drive/My Drive/NewData/7113.MY/Unzipped/'  #topglove
#unzipped_dir = '/content/drive/My Drive/NewData/6947.MY/Unzipped/'  #digi
#unzipped_dir = '/content/drive/My Drive/NewData/8869.MY/Unzipped/' #pmetal
#unzipped_dir = '/content/drive/My Drive/NewData/7160.MY/Unzipped/' #penta

li = [];
filePrefix = "quotemovements_";
dateLength = len("20191106"); 

# Get all the files in unzipped folder in ascending order
print( sorted( os.listdir(unzipped_dir) )) 
for item in sorted( os.listdir(unzipped_dir) ): # loop through items in dir
  startIndex = item.find("_", len( filePrefix) )
  date = item[startIndex+1:startIndex+dateLength+1]
  filepath = os.path.abspath(os.path.join(unzipped_dir, item))
  df = pd.read_csv( filepath)
  df["Date"] = date
  df = df[::-1]   # Reverse the order of data frame
  li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)
frame.head()
frame['Datetime'] = pd.to_datetime(frame["Date"] + " " + frame["Time"])
frame["Index"]    = range(len(frame))
#frame.sort_values( by = ['Datetime'], inplace = True )
frame = frame.set_index('Index')
frame.to_csv( "/content/test.csv" );

In [0]:
# Get data for certain day
allDates = frame["Date"].unique()
print( len( allDates) )
processedData = {};
data = {};
for date in allDates:
  subset = frame["Date"] == date
  dailyData = frame[subset]
  dailyData.loc[:,'Last Done Price'] = dailyData.loc[:,'Last Done Price'].ffill()
  dailyData.fillna( 0, inplace = True )
  dailyData.to_csv( "/content/feff.csv" );

  # Get the unique buy & sell prices of the day
  buyPrice = dailyData[ dailyData['Buy Price'] > 0.0 ]['Buy Price'].unique()
  buyPriceList = {}
  for price in buyPrice:
    buyPriceList[price] = 0.0

  sellPrice = dailyData[ dailyData['Sell Price'] > 0.0 ]['Sell Price'].unique()
  sellPriceList = {}
  for price in sellPrice:
    sellPriceList[price] = 0.0

  dailyData['CurrentSupply']  = 0.0
  dailyData['CurrentDemand']  = 0.0
  dailyData["TotalDemand"]    = 0.0  
  dailyData["TotalSupply"]    = 0.0
  dailyData["BuyUpTrade"]     = 0.0
  dailyData["SellDownTrade"]  = 0.0
  dailyData["PreOpenTrade"]   = 0.0  
  
  buyUpTrade = 0.0;
  sellDownTrade = 0.0;
  preOpenTrade  = 0.0;

  for index, row in dailyData.iterrows():
      buyPrice     = row["Buy Price"]
      buyVolChg    = row["Buy Vol Change"]
      buyVol       = row["Buy Vol"]
      sellVolChg   = row["Sell Vol Change"]
      sellPrice    = row["Sell Price"]
      sellVol      = row["Sell Vol"]
      tradeVol     = row["Last Done Vol"]
      tradePrice   = row["Last Done Price"]
      tradeType    = row["Type"]
      
      currentDemand = buyPrice * buyVol
      currentSupply = sellPrice * sellVol
      price = tradeVol * tradePrice;
      if(tradeType == "Preopen"):
        preOpenTrade = price + preOpenTrade
      elif( tradeType == "Buy Up" ):
        buyUpTrade = price + buyUpTrade
      elif( tradeType == "Sell Down"):
        sellDownTrade = price + sellDownTrade

      # Calculate accumulated total buy / sell 
      if( row["Buy Price"] in buyPriceList ):
        buyPriceList[row["Buy Price"]] = currentDemand
      if( row["Sell Price"] in sellPriceList ):
        sellPriceList[row["Sell Price"]] = currentSupply

      accumulatedDemand = sum(buyPriceList.values())
      accumulatedSupply = sum(sellPriceList.values())

      dailyData.loc[index, 'CurrentSupply']    = currentSupply
      dailyData.loc[index, 'CurrentDemand']    = currentDemand
      dailyData.loc[index, 'TotalDemand']      = accumulatedDemand    
      dailyData.loc[index, 'TotalSupply']      = accumulatedSupply
      dailyData.loc[index, "BuyUpTrade"]       = buyUpTrade
      dailyData.loc[index, "SellDownTrade"]    = sellDownTrade
      dailyData.loc[index, "PreOpenTrade"]     = preOpenTrade
      dailyData.loc[index, "TotalTrade"]       = buyUpTrade + sellDownTrade + preOpenTrade

  dailyData2 = dailyData[['Datetime', 'TotalDemand', 'TotalSupply', 'TotalTrade', 'BuyUpTrade', 'SellDownTrade', 'PreOpenTrade', 'Last Done Price', 'Time']].copy()
  dailyData2 = dailyData2.set_index('Datetime')
  aggregatedData = dailyData2.resample( '5T' ).last() # get last record for every 5 minutes
  aggregatedData.dropna(subset = ["Time"], inplace=True) # drop empty row  from trade break
  data[date]   = dailyData2
  processedData[date] = aggregatedData
  #aggregatedData.to_csv("/content/" + date +".csv")


In [0]:
test = {};
import numpy as np
from decimal import Decimal
#Google trend
colnames=['Date', 'Popularity'] 

gtfilepath = "/content/drive/My Drive/GoogleTrend/MAYBANK.csv"
gtfilepath = "/content/drive/My Drive/GoogleTrend/REVENUE.csv"
gtfilepath = "/content/drive/My Drive/GoogleTrend/HIBISCS.csv"
gtfilepath = "/content/drive/My Drive/GoogleTrend/AIRASIA.csv"
gtfilepath = "/content/drive/My Drive/GoogleTrend/EKOVEST.csv"
#gtfilepath = "/content/drive/My Drive/GoogleTrend/FGV.csv"
#gtfilepath = "/content/drive/My Drive/GoogleTrend/GENM.csv"
#gtfilepath = "/content/drive/My Drive/GoogleTrend/JAKS.csv"
#gtfilepath = "/content/drive/My Drive/GoogleTrend/TOPGLOVE.csv"
#gtfilepath = "/content/drive/My Drive/GoogleTrend/DIGI.csv"
#gtfilepath = "/content/drive/My Drive/GoogleTrend/PMETAL.csv"
#gtfilepath = "/content/drive/My Drive/GoogleTrend/PENTA.csv"


googleTrend = pd.read_csv(gtfilepath, skiprows = 3, names = colnames )
googleTrend['Index'] = googleTrend['Date'].apply(lambda x: x.replace("-","") )

for key,value in data.items():
  aggregatedData = value.resample( '60T' ).last() # get last record for every 1hour
  aggregatedData['Popularity'] = googleTrend[googleTrend['Index'] == key]['Popularity'].iloc[0]
  aggregatedData['TotalTrade'].replace( 0.00, np.nan)
  aggregatedData.dropna(subset = ["TotalTrade"], inplace=True) # drop empty row  from trade break
  aggregatedData = aggregatedData.loc[ ~np.isclose( aggregatedData['Last Done Price'], 0.00 ) ]
  print(aggregatedData)
  test[key] = aggregatedData

# Concatenate dataset
hourdata = pd.concat( test, axis = 0 );

# Export to csv
#finalfilepath = "/content/drive/My Drive/FinalizedData/MAYBANK.csv"
#finalfilepath = "/content/drive/My Drive/FinalizedData/REVENUE.csv"
#finalfilepath = "/content/drive/My Drive/FinalizedData/HIBISCS.csv"
#finalfilepath = "/content/drive/My Drive/FinalizedData/AIRASIA.csv"
#finalfilepath = "/content/drive/My Drive/FinalizedData/EKOVEST.csv"
##finalfilepath = "/content/drive/My Drive/FinalizedData/FGV.csv"
#finalfilepath = "/content/drive/My Drive/FinalizedData/GENM.csv"
#finalfilepath = "/content/drive/My Drive/FinalizedData/JAKS.csv"
#finalfilepath = "/content/drive/My Drive/FinalizedData/TOPGLOVE.csv"
#finalfilepath = "/content/drive/My Drive/FinalizedData/DIGI.csv"
#finalfilepath = "/content/drive/My Drive/FinalizedData/PMETAL.csv"
#finalfilepath = "/content/drive/My Drive/FinalizedData/PENTA.csv"

hourdata.to_csv(finalfilepath)